In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/youtoxic_english_1000.csv")

print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.head()


In [ ]:
# 1. Revisar columnas con solo ceros o sin representación significativa
etiquetas = [
    "IsToxic", "IsAbusive", "IsThreat", "IsProvocative", "IsObscene",
    "IsHatespeech", "IsRacist", "IsNationalist", "IsSexist", "IsHomophobic",
    "IsReligiousHate", "IsRadicalism"
]

# Mostrar cuántas veces aparece "True" en cada etiqueta
print("Etiquetas activas:")
print(df[etiquetas].sum().sort_values())

# 2. Eliminar columnas sin representación (sum == 0 o sum == 1)
etiquetas_utiles = [col for col in etiquetas if df[col].sum() > 1]
print("\nEtiquetas que conservaremos:", etiquetas_utiles)

# 3. Eliminar columnas irrelevantes para el modelado
columnas_irrelevantes = ["CommentId", "VideoId"]
df.drop(columns=columnas_irrelevantes, inplace=True)

# 4. (Opcional) Eliminar duplicados por texto
df.drop_duplicates(subset="Text", inplace=True)

# 5. Dejar el DataFrame con solo las columnas útiles
columnas_utiles = ["Text"] + etiquetas_utiles
df = df[columnas_utiles]

# 6. Confirmar
print(f"\nDataset limpio → Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.head()


### Limpieza previa del dataset

Antes de aplicar el preprocesamiento textual, revisamos las columnas del dataset:

- Eliminamos etiquetas que **no tienen representación suficiente** (por ejemplo, `IsHomophobic`, `IsRadicalism`) ya que no aportarían valor al modelo.
- Quitamos columnas irrelevantes como `CommentId` y `VideoId`.
- Eliminamos posibles duplicados exactos en los comentarios (`Text`).

Esto nos permite trabajar sobre un dataset más limpio, centrado en los comentarios y en las etiquetas que tienen información útil. De este modo, evitamos introducir ruido o clases vacías en el modelo.


In [ ]:
import spacy
import re

# Cargar modelo de inglés
nlp = spacy.load("en_core_web_sm")

# Función para limpiar y lematizar texto
def preprocesar_texto(texto):
    # 1. Eliminar URLs, símbolos especiales y números
    texto = re.sub(r"http\S+|www\S+|[^a-zA-Z\s]", "", texto.lower())
    
    # 2. Procesar con SpaCy
    doc = nlp(texto)

    # 3. Eliminar stopwords y obtener lemas
    tokens_limpios = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    return " ".join(tokens_limpios)


In [ ]:
# Aplicar limpieza sobre una copia del texto
df["CleanText"] = df["Text"].apply(preprocesar_texto)

# Ver resultado en 3 ejemplos
df[["Text", "CleanText"]].sample(3)


### Preprocesamiento del texto con SpaCy

En este paso limpiamos y normalizamos los comentarios para que puedan ser procesados por modelos de machine learning. Para ello utilizamos **SpaCy**, una librería especializada en procesamiento de lenguaje natural.

Nuestra función `preprocesar_texto()` realiza las siguientes tareas:

1. **Elimina elementos innecesarios** del texto original:
   - URLs, símbolos, números y puntuación.
   - Convierte todo a minúsculas.

2. **Tokeniza** (divide en palabras) y procesa cada palabra con SpaCy:
   - Elimina las **palabras vacías** (*stopwords*) como "the", "you", "and".
   - Obtiene la **forma base** (*lema*) de cada palabra, por ejemplo:
     - "running" → "run"
     - "was" → "be"
     - "better" → "good"

3. **Reconstruye el texto limpio**, que usaremos como entrada para vectorización y modelado.

Este preprocesamiento mejora significativamente la calidad de las características que extraeremos del texto, ya que reduce el ruido y normaliza el lenguaje.


In [ ]:
# Esto puede tardar unos segundos la primera vez (1.000 textos)
df["CleanText"] = df["Text"].apply(preprocesar_texto)

# Comprobar algunos ejemplos de texto original vs limpio
df[["Text", "CleanText"]].sample(5, random_state=42)
